In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn import metrics

df=pd.read_csv("firstTry.csv")
df

,predictor,rating_chooses_appropriate_action,rating_commits_to_action,rating_gathers_information,rating_identifies_issues_opportunities,rating_interprets_information,rating_involves_others,rating_decision_making_final_score
0,"Hi Tracy, This sounds very ionteresting, prio...",2.0,2,2,2.0,2,3.0,2
1,"Hi Tracy, I am happy to see we have a self mo...",4.0,3,4,3.0,4,2.0,6
2,J.J please can you work with her for two mor...,2.0,2,2,2.0,2,2.0,2
3,"Hi Paxton Let me talk to Debby first, before...",3.0,3,3,2.0,2,2.0,3
4,JJ HR along with myself and the Final Assem...,3.0,3,2,3.0,2,2.0,3
...,...,...,...,...,...,...,...,...
1461,"Dear Tracey,I wanted to take the time to than...",2.0,2,2,3.0,3,2.0,2
1462,Hi Tracy - thanks for your note and for shari...,4.0,4,3,4.0,4,4.0,6
1463,"Dear Tracey,Thanks for sharing the data.We do...",3.0,3,2,3.0,2,2.0,2
1464,"Hi Tracy, Thanks for sharing customer satisfa...",4.0,3,3,4.0,3,4.0,6


In [ ]:
# Criterion 2 model

#Note: Once you find a satisfactory cross-validated score, you don't need to split the data, just run the model on everything
#and predict the validate.csv on the second chunk
y=df['rating_commits_to_action']
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)

#count_vectorizer = CountVectorizer(stop_words='english')
#count_train = count_vectorizer.fit_transform(X_train.values)
#count_test = count_vectorizer.transform(X_test.values)

#Converting words to numbers
#Note: try other methods if you have time, like sentiment analysis, bag of words, or quanteda. 
#Note: You might need to change max_features if it gives you an error in the next chunk, when predicting the validate dataset
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=12000)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

#Hyperparameter tuning
#Note: use less parameters or make the ranges shorter if it's taking too long
#param_grid = {
    #'hidden_layer_sizes': [(10,30,10),(20,)],
    #'activation': ['tanh', 'relu'],
    #'solver': ['sgd', 'adam'],
    #'alpha': [0.0001, 0.05],
    #'learning_rate': ['constant','adaptive']}
    
param_grid = {'activation':['tanh'],
             'alpha': [0.0001, 0.01],
             'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
             'learning_rate': ['constant'],
             'solver': ['sgd']}

#Fitting model
#Note: Hillclimbing methods could be used instead of gridsearch. This might be quicker. 
#Note: randomizedSearchCV() could be used instead of GridSearchCV() to make the algorithm quicker.
#Note: try other models, like neural network classifier. We prefer a classification algorithm instead of regression. 
grid = GridSearchCV(MLPRegressor(max_iter=500), 
                    param_grid=param_grid, cv=5, verbose=3, scoring=['neg_mean_squared_error'])
    
grid.fit(tfidf_train, y_train)
pred2 = grid.predict(tfidf_test)

#print("Best parameters:", grid.best_params_)
#Note: If you're using a classifier, you're going to have to change the metric to be the AUC or accuracy instead of the RMSE or r2
print("Lowest RMSE: ", (-grid.best_score_)**(1/2.0))
r2_score(y_test, pred2)
#Use this for classification

#print("test-set score: {:.3f}".format(grid.score(X_test, y_test)))
#from sklearn.metrics import roc_auc_score
#roc_auc_score(y_test, pred1)
#metrics.accuracy_score(y_test, pred1)

In [ ]:
#This chunk is predicting the test csv
validation=pd.read_csv("validate.csv")
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=12000)
count= tfidf_vectorizer.fit_transform(validation['predictor'])
pred2 = grid.predict(count)
pred2
#Probably need to convert pred2 into a df
pred2.to_csv(r'criterion1_NNRegressor.csv', index=False)
#OR
np.savetxt("criterion1_NNRegressor.csv", pred2, delimiter = ",", header="rating_chooses_appropriate_action", fmt="%d", comments="")

In [ ]:
######### NOW XGBOOST!!!!! ########################

In [2]:
# Criterion 2 model

#Note: Once you find a satisfactory cross-validated score, you don't need to split the data, just run the model on everything
#and predict the validate.csv on the second chunk
y=df['rating_commits_to_action']
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)

#count_vectorizer = CountVectorizer(stop_words='english')
#count_train = count_vectorizer.fit_transform(X_train.values)
#count_test = count_vectorizer.transform(X_test.values)

#Converting words to numbers
#Note: try other methods if you have time, like sentiment analysis, bag of words, or quanteda. 
#Note: You might need to change max_features if it gives you an error in the next chunk, when predicting the validate dataset
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=12000)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

#Hyperparameter tuning
#Note: use less parameters or make the ranges shorter if it's taking too long
param_grid = {'max_depth':range(1, 7),
                'learning_rate':np.arange(0.005, 0.3, 0.07),
                 'n_estimators':np.arange(100, 2000, 200),
                 'gamma':np.arange(0, 0.06, 0.01),
                 'subsample':np.arange(0.5, 1, 0.1),
                 'colsample_bytree':np.arange(0.5, 1, 0.1)}

#Fitting model
#Note: Hillclimbing methods could be used instead of gridsearch. This might be quicker. 
#Note: randomizedSearchCV() could be used instead of GridSearchCV() to make the algorithm quicker.
#Note: try other models, like neural network classifier. We prefer a classification algorithm instead of regression. 
grid = GridSearchCV(xgb.XGBRegressor(random_state=0), 
                    param_grid=param_grid, cv=5, verbose=3, scoring='neg_mean_squared_error')
    
grid.fit(tfidf_train, y_train)
pred2 = grid.predict(tfidf_test)

print("Best parameters:", grid.best_params_)
#Note: If you're using a classifier, you're going to have to change the metric to be the AUC or accuracy instead of the RMSE or r2
print("Lowest RMSE: ", (-grid.best_score_)**(1/2.0))
r2_score(y_test, pred2)
#Use this for classification
#metrics.accuracy_score(y_test, pred)

Fitting 5 folds for each of 45000 candidates, totalling 225000 fits
[CV 1/5] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, n_estimators=100, subsample=0.5;, score=-2.787 total time=   0.2s
[CV 2/5] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, n_estimators=100, subsample=0.5;, score=-2.776 total time=   0.1s
[CV 3/5] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, n_estimators=100, subsample=0.5;, score=-2.797 total time=   0.1s
[CV 4/5] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, n_estimators=100, subsample=0.5;, score=-2.852 total time=   0.1s
[CV 5/5] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, n_estimators=100, subsample=0.5;, score=-2.804 total time=   0.1s
[CV 1/5] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, n_estimators=100, subsample=0.6;, score=-2.783 total time=   0.1s
[CV 2/5] END colsample_bytree=0.5, gamma=0.0, learning_r

In [ ]:
#This chunk is predicting the test csv
validation=pd.read_csv("validate.csv")
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=12000)
count= tfidf_vectorizer.fit_transform(validation['predictor'])
pred2 = grid.predict(count)
pred2
#Probably need to convert pred2 into a df
pred2.to_csv(r'criterion1_XGBRegressor.csv', index=False)
#OR
np.savetxt("criterion1_XGBRegressor.csv", pred2, delimiter = ",", header="rating_commits_to_action", fmt="%d", comments="")